# Ansible Einführung

Warum eigentlich Ansible und nicht bash-Skripte?

In welchem Verzeichnis befinden wir uns?

In [1]:
pwd

/home/vagrant/ais/hands-on-ansible/01-first-steps


##  Vorbereitung der Container

Zunächst alle weg werfen und neu aufsetzen.

In [2]:
for i in ais-bashy db web1 web2 playbooks; do
    sudo lxc-destroy --name ${i} --force
done
ansible-playbook -i localhost, ../00-home-base/setup-lxc.yml
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP
ssh root@${AIS_BASHY_IP} uptime

Destroyed container ais-bashy
Destroyed container db
Destroyed container web1
Destroyed container web2
Destroyed container playbooks
 ____________
< PLAY [all] >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ________________________________
< TASK [Init AIS lxc containers] >
 --------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [localhost]
 ____________
< PLAY RECAP >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

localhost                  : ok=1    changed=1    unreachable=0    failed=0   

10.0.3.207
 13:17:30 up 1 day,  6:26,  0 users,  load average: 1,49, 0,65, 0,48


## Inhalt des Verzeichnisses

Falls `tree` nicht installiert sein sollte, dann installieren und nochmal ausführen.

In [3]:
tree . || (sudo apt-get update && sudo apt-get install --yes tree && tree .)

.
├── 01_Ansible_vs_Bashscript.ipynb
├── Bash-Problems
├── config
│   ├── index.html
│   ├── mywebsite.conf
│   └── nginx.conf
├── install-nginx.retry
├── install-nginx.sh
└── install-nginx.yml

1 directory, 8 files


## Das Shell Skript

In [4]:
cat install-nginx.sh

#!/bin/bash
set -euo pipefail
pwd
cd "${BASH_SOURCE%/*}" || exit 
pwd

# Install nginx.
apt-get update
apt-get install -y nginx sshguard nano curl # vim

# Ensure website directory exists
mkdir -p /var/www/mywebsite

# Copy nginx config files (main config and vhost config)
cp /tmp/config/nginx.conf /etc/nginx/nginx.conf
chown root:root /etc/nginx/nginx.conf
chmod 0644 /etc/nginx/nginx.conf

cp /tmp/config/mywebsite.conf /etc/nginx/conf.d/mywebsite.conf
chown root:root /etc/nginx/conf.d/mywebsite.conf
chmod 0644 /etc/nginx/conf.d/mywebsite.conf

# Copy our website files
cp /tmp/config/index.html /var/www/mywebsite/
chown root:root /var/www/mywebsite/index.html
chmod 0644 /var/www/mywebsite/index.html

# Remove default nginx vhost configuration
rm /etc/nginx/sites-enabled/default

# Start nginx at boot + make sure it's running right now.
systemctl restart nginx.service
systemctl enable nginx.service

# On a machine with systemd, you'd use these instead
# systemctl enable nginx
# systemct

Wegen `set -euo pipefail` siehe

https://vaneyckt.io/posts/safer_bash_scripts_with_set_euxo_pipefail/

Wegen `cd "${BASH_SOURCE%/*}" || exit` siehe

http://mywiki.wooledge.org/BashFAQ/028

In [5]:
ssh root@${AIS_BASHY_IP} install-nginx.sh

bash: install-nginx.sh: Befehl nicht gefunden


: 127

In [6]:
scp -r install-nginx.sh config/ root@${AIS_BASHY_IP}:/tmp/

install-nginx.sh                              100% 1017     1.0KB/s   00:00    
mywebsite.conf                                100%  455     0.4KB/s   00:00    
index.html                                    100%  976     1.0KB/s   00:00    
nginx.conf                                    100%  745     0.7KB/s   00:00    


In [7]:
ssh root@${AIS_BASHY_IP} ls /tmp/

config
install-nginx.sh


In [8]:
ssh root@${AIS_BASHY_IP} /tmp/install-nginx.sh

/root
/tmp
OK:1 http://security.ubuntu.com/ubuntu xenial-security InRelease
OK:2 http://archive.ubuntu.com/ubuntu xenial InRelease
OK:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease
Paketlisten werden gelesen...
Paketlisten werden gelesen...
Abhängigkeitsbaum wird aufgebaut....
Statusinformationen werden eingelesen....
Die folgenden zusätzlichen Pakete werden Installiert
  fontconfig-config fonts-dejavu-core geoip-database iptables libfontconfig1
  libfreetype6 libgd3 libgeoip1 libicu55 libjbig0 libjpeg-turbo8 libjpeg8
  libnfnetlink0 libtiff5 libvpx3 libx11-6 libx11-data libxau6 libxcb1
  libxdmcp6 libxml2 libxpm4 libxslt1.1 nginx-common nginx-core sgml-base
  xml-core
Vorgeschlagene Pakete:
  libgd-tools geoip-bin spell fcgiwrap nginx-doc ssl-cert sgml-base-doc
  debhelper
Die folgenden NEUEN Pakete werden installiert:
  curl fontconfig-config fonts-dejavu-core geoip-database iptables
  libfontconfig1 libfreetype6 libgd3 libgeoip1 libicu55 libjbig0
  libjpeg-turbo8 libjpe

Entpacken von nano (2.5.3-2ubuntu2) ...
Vormals nicht ausgewähltes Paket xml-core wird gewählt.
Vorbereitung zum Entpacken von .../xml-core_0.13+nmu2_all.deb ...
Entpacken von xml-core (0.13+nmu2) ...
Vormals nicht ausgewähltes Paket curl wird gewählt.
Vorbereitung zum Entpacken von .../curl_7.47.0-1ubuntu2.2_amd64.deb ...
Entpacken von curl (7.47.0-1ubuntu2.2) ...
Vormals nicht ausgewähltes Paket fonts-dejavu-core wird gewählt.
Vorbereitung zum Entpacken von .../fonts-dejavu-core_2.35-1_all.deb ...
Entpacken von fonts-dejavu-core (2.35-1) ...
Vormals nicht ausgewähltes Paket fontconfig-config wird gewählt.
Vorbereitung zum Entpacken von .../fontconfig-config_2.11.94-0ubuntu1.1_all.deb ...
Entpacken von fontconfig-config (2.11.94-0ubuntu1.1) ...
Vormals nicht ausgewähltes Paket libfreetype6:amd64 wird gewählt.
Vorbereitung zum Entpacken von .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Entpacken von libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Vormals nicht ausgewähltes Paket libfon

In [12]:
echo https://${AIS_BASHY_IP}

https://10.0.3.174


In [9]:
cat Bash-Problems

Problems

-We're running this on the target machine automatically with Ansible. With bash:
	-scp script + resources
	-ssh to remote machine; run
	-clean up

0. Hardcoded IP in mywebsite.conf
1. We want to comment out 'vim' -- it takes up too much space for our server
2. We want to run this on 1000 servers -- only our 'web' servers, though.
3. We want to add a second site, stop hardcoding everything, and re-use a single configuration file
4. We also want to set up a database server, which is logically a part of our web application, in the same 'deployment' step.
5. We want to enforce a consistent structure between scripts



##  Nginx mit Ansible aufsetzen

Zunächst den Container neu aufsetzen.

In [10]:
sudo lxc-destroy --name ais-bashy --force

Destroyed container ais-bashy


Die Container einrichten und die Umgebungsvariable setzen.

In [11]:
ansible-playbook -i localhost, ../00-home-base/setup-lxc.yml
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP
ssh root@${AIS_BASHY_IP} uptime

 ____________
< PLAY [all] >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ________________________________
< TASK [Init AIS lxc containers] >
 --------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [localhost]
 ____________
< PLAY RECAP >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

localhost                  : ok=1    changed=1    unreachable=0    failed=0   

10.0.3.174
 06:31:46 up 1 day, 23:40,  0 users,  load average: 0,60, 0,23, 0,09


In [15]:
sudo lxc-ls --fancy

NAME      STATE   AUTOSTART GROUPS IPV4       IPV6 
ais-bashy RUNNING 0         -      10.0.3.174 -    
db        RUNNING 0         -      10.0.3.88  -    
playbooks RUNNING 0         -      10.0.3.231 -    
web1      RUNNING 0         -      10.0.3.86  -    
web2      RUNNING 0         -      10.0.3.62  -    


In [14]:
ansible-playbook -i $AIS_BASHY_IP, -u root install-nginx.yml

 ____________
< PLAY [all] >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ________________________
< TASK [Gathering Facts] >
 ------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

ok: [10.0.3.174]
 _______________________
< TASK [Install nginx.] >
 -----------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [10.0.3.174] => (item=nginx)
changed: [10.0.3.174] => (item=nano)
changed: [10.0.3.174] => (item=sshguard)
changed: [10.0.3.174] => (item=curl)
 _________________________________
< TASK [Create website directory] >
 ---------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [10.0.3.174]
 _______

In [ ]:
cat install-nginx.yml

In [ ]:
echo http://$AIS_BASHY_IP